# Weather Data Preparation 

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

https://www.wunderground.com/history/daily/us/il/chicago/KMDW/date/2017-8-4

In [2]:
# read data
df = pd.read_csv('data/raw/weather_hourly.csv', index_col=0)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10334 entries, 0 to 10333
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Time            10334 non-null  object
 1   Temperature     10334 non-null  object
 2   Dew Point       10334 non-null  object
 3   Humidity        10334 non-null  object
 4   Wind            10332 non-null  object
 5   Wind Speed      10334 non-null  object
 6   Wind Gust       10334 non-null  object
 7   Pressure        10334 non-null  object
 8   Precip.         10334 non-null  object
 9   Condition       10334 non-null  object
 10  date            10334 non-null  object
 11  boolean_column  10334 non-null  bool  
dtypes: bool(1), object(11)
memory usage: 978.9+ KB


In [12]:
# reduce dataframe to important columns
df.drop(['Dew Point', 'Wind', 'Wind Gust', 'Pressure', 'boolean_column'], axis=1, inplace=True)

In [7]:
df['Time'].unique()

array(['12:51 AM', '1:36 AM', '1:51 AM', '2:51 AM', '3:51 AM', '4:36 AM',
       '4:51 AM', '5:51 AM', '6:51 AM', '7:51 AM', '8:51 AM', '9:51 AM',
       '9:57 AM', '10:07 AM', '10:12 AM', '10:44 AM', '10:49 AM',
       '10:51 AM', '10:58 AM', '11:09 AM', '11:51 AM', '12:51 PM',
       '1:34 PM', '1:51 PM', '2:14 PM', '2:51 PM', '3:06 PM', '3:43 PM',
       '3:51 PM', '4:00 PM', '4:51 PM', '5:09 PM', '5:19 PM', '5:33 PM',
       '5:51 PM', '6:51 PM', '7:06 PM', '7:51 PM', '8:06 PM', '8:30 PM',
       '8:51 PM', '9:20 PM', '9:43 PM', '9:51 PM', '10:51 PM', '11:51 PM',
       '1:44 AM', '3:13 AM', '4:39 AM', '5:47 AM', '6:04 AM', '6:18 AM',
       '7:25 AM', '7:49 AM', '8:16 AM', '8:28 AM', '9:09 AM', '9:21 AM',
       '10:17 AM', '11:30 AM', '1:25 PM', '2:34 PM', '3:53 PM', '4:04 PM',
       '4:30 PM', '5:04 PM', '5:07 PM', '6:14 PM', '6:23 PM', '6:48 PM',
       '7:34 PM', '8:10 PM', '8:32 PM', '8:41 PM', '10:05 PM', '10:49 PM',
       '5:05 AM', '5:15 AM', '5:30 AM', '6:16 AM', '6:35 

In [14]:
# clear units + convert formats 
df['Temperature'] = df['Temperature'].str.replace(r'°F', '').astype(int)
df['Humidity'] = df['Humidity'].str.replace(r'°%', '').astype(int)
df['Wind Speed'] = df['Wind Speed'].str.replace(r'°mph', '').astype(int)
df['Precip.'] = df['Precip.'].str.replace(r'°in', '')
df['Precip.'] = df['Precip.'].str.split().str.join(' ').astype(float)

### create datetime column

In [15]:
# convert raw data to datetime format 
df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%d_%m').replace(year=2014).date()) # date column 
df['Time'] = df['Time'].apply(lambda x: pd.to_datetime(x, format="%I:%M %p").time()) # time column 

# merge column date and time to datetime and drop unused columns
df['datetime'] = df.apply(lambda x: datetime.combine(x['date'], x['Time']), axis=1)
df.drop(['date', 'Time'], axis=1, inplace=True)

# set index
df = df.set_index('datetime')

### categorize and encode conditions

In [17]:
# inspect categorical values
df['Condition'].value_counts() 

# subtract 'windy' from string in 'condition' as information is covered by wind speed column
df['cond_new'] = df['Condition'].str.replace(' / Windy', '')

# TODO check if needed 
# check precip vs rain feature  -> precip column does not capture occurence of rain like condition column
#len(df[(df["Precip."] >= 0.01)]) 
#len(df.where[df['Condition' == 'Rain'] & df['Condition' == 'Light Rain']])

# simplify conditions by merging similar categories 
df['cond_new'] = df['cond_new'].str.replace('Snow and Sleet', 'Snow').replace('Rain and Snow', 'Snow').replace('Light Snow', 'Snow') # simplify 'Light Snow' + 'Snow and Sleet' + 'Rain and Snow' to 'Snow' 
df['cond_new'] = df['cond_new'].str.replace('Haze', 'Fog').replace('Patches of Fog', 'Fog') # simplify 'haze' and 'patches of fog' to fog
df['cond_new'] = df['cond_new'].str.replace('Light Rain with Thunder', 'Thunder') # simplify 'Light Rain with Thunder' to 'Thunder'
df['cond_new'] = df['cond_new'].str.replace('Heavy T-Storm', 'T-Storm') # simplify 'Heavy T-Storm' to 'T-Storm'

# ordinal encoding of features rain + clouds 
df['cloud'] = df['cond_new'].replace('Cloudy', 3).replace('Mostly Cloudy', 2).replace('Partly Cloudy', 1).apply(pd.to_numeric, errors='coerce') # ordinal encoding of clouds
df['rain'] = df['cond_new'].replace('Heavy Rain', 4).replace('Rain', 3).replace('Light Rain', 2).replace('Light Drizzle', 1).apply(pd.to_numeric, errors='coerce') # ordinal encoding of rain
df[['cloud', 'rain']] = df[['cloud', 'rain']].fillna(0) 

# one hot encoding of nominal features
nominal_features = df['cond_new'].replace(dict.fromkeys(['Cloudy','Mostly Cloudy','Partly Cloudy','Heavy Rain', 'Rain', 'Light Rain', 'Light Drizzle'], np.nan)) # subtract ordinal features
one_hot = pd.get_dummies(nominal_features) # one-hot encoding 
df = df.join(one_hot) # join encoded variables 
df.drop(['Condition', 'cond_new'], axis=1, inplace=True) # drop unused column 

### resample hourly 

In [19]:
# number of observations in raw dataset and of expected hourly observations compared
print('number of actual observations: ' + str(len(df)))
print('number of expected obersavtions: ' + str(365*24))

number of actual observations: 10344
number of expected obersavtions: 8760


In [21]:
# resample hourly 
df = df.resample('H').last()
df.dtypes # check new dtypes
# TODO convert dtypes

Temperature    float64
Humidity       float64
Wind Speed     float64
Precip.        float64
cloud          float64
rain           float64
Fair           float64
Fog            float64
Snow           float64
T-Storm        float64
Thunder        float64
dtype: object

In [22]:
# check for NaN
df[df.isna().any(axis=1)]

#interpolate metric values and temperature 
df[['Temperature', 'Humidity', 'Wind Speed', 'Precip.']] = df[['Temperature', 'Humidity', 'Wind Speed', 'Precip.']].interpolate(method='linear', axis=0)
df[['Temperature', 'Humidity', 'Wind Speed']] = df[['Temperature', 'Humidity', 'Wind Speed']].round(decimals=0)
df['Precip.'] = df['Precip.'].round(decimals=1)

# interpolate binary + ordinal features 
df[['cloud', 'rain', 'Fair', 'Fog', 'Snow', 'T-Storm', 'Thunder']] = df[['cloud', 'rain', 'Fair', 'Fog', 'Snow', 'T-Storm', 'Thunder']].interpolate(method='pad', axis=0)


df[df.isna().any(axis=1)]

,Temperature,Humidity,Wind Speed,Precip.,cloud,rain,Fair,Fog,Snow,T-Storm,Thunder
datetime,,,,,,,,,,,


In [23]:
# convert fahrenheit to celsius
def celsius(x):
    x = (x-32)*5/9
    return float(x)

df['Temperature'] = df['Temperature'].apply(celsius).round(decimals=1)

In [24]:
#convert miles per hour to kilometers per hour
def kmh(x):
    x = x * 1.609
    return float(x)

df['Wind Speed'] = df['Wind Speed'].apply(kmh).round(decimals = 2)

In [25]:
#convert inches to centimeters
def mm(x):
    x = x * 25.4
    return float(x)

df['Precip.'] = df['Precip.'].apply(mm).round(decimals = 2)

In [26]:
#TODO normalize data 
df

,Temperature,Humidity,Wind Speed,Precip.,cloud,rain,Fair,Fog,Snow,T-Storm,Thunder
datetime,,,,,,,,,,,
2014-01-01 00:00:00,-10.0,84.0,11.26,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-01-01 01:00:00,-8.9,84.0,9.65,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-01-01 02:00:00,-8.3,84.0,11.26,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-01-01 03:00:00,-7.8,84.0,11.26,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-01-01 04:00:00,-7.2,85.0,9.65,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2014-12-31 19:00:00,-7.2,47.0,32.18,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2014-12-31 20:00:00,-7.2,50.0,25.74,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2014-12-31 21:00:00,-6.7,48.0,27.35,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [27]:
df.describe()

,Temperature,Humidity,Wind Speed,Precip.,cloud,rain,Fair,Fog,Snow,T-Storm,Thunder
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000
mean,9.857169,66.552169,16.369316,0.092785,1.596918,0.100000,0.216096,0.012215,0.021575,0.002283,0.005251
std,12.508246,16.208322,8.378385,0.942619,1.239994,0.436177,0.411604,0.109849,0.145301,0.047730,0.072278
min,-25.600000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,55.000000,11.260000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,11.100000,68.000000,14.480000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,20.600000,79.000000,20.920000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,33.900000,100.000000,59.530000,27.940000,3.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [28]:
df.to_pickle("data/weather_data_hourly_1.pickle")

### Create matching aggregations for resolutions of 4h and 6h

### Aggregations:

- Temperature: mean
- Humidity: mean
- Wind Speed: mean
- Precip.: mean
- cloud: 1 if it was party cloudy, 2 if mostly cloudy and 3 if cloudy the majority of the hours, else 0
- rain: 1 if it drizzled lightly, 2 if it rained lightly, 2 if it rained and 3 if it rained heavy the majority of hours, else 0
- Fair: 1 if it there was Fair during the majority of hours, else 0
- Fog: 1 if it there was Fog during the majority of hours, else 0
- Snow: 1 if it there was Snow during the majority of hours, else 0
- T-Storm: 1 if it there was thunder storm during the majority of hours, else 0
- Thunder: 1 if it there was thunder during the majority of hours, else 0

In [29]:
def map_values(mean_var):
    if 0.5 < mean_var < 1.5:
        return 1
    elif 1.5 < mean_var < 2.5:
        return 2
    elif 2.5 < mean_var < 3.5:
        return 3
    else:
        return 0

In [48]:
def aggregate_weather_data(df, time_interval):
    # Calculate the estimated resolution based on the time_interval
    resolution = time_interval
    # Resample the data based on the calculated resolution
    aggregated_df = df.resample(f'{resolution}H').mean()
    

    aggregated_df['cloud'] = aggregated_df['cloud'].apply(map_values)
    aggregated_df['rain'] = aggregated_df['rain'].apply(map_values)
    aggregated_df['Fair'] = aggregated_df['Fair'].apply(map_values)
    aggregated_df['Fog'] = aggregated_df['Fog'].apply(map_values)
    aggregated_df['Snow'] = aggregated_df['Snow'].apply(map_values)
    aggregated_df['T-Storm'] = aggregated_df['T-Storm'].apply(map_values)
    aggregated_df['Thunder'] = aggregated_df['Thunder'].apply(map_values)
    return aggregated_df
    

In [33]:
def calculate_resolution(df, time_interval):
    mean_time_difference = df.index.to_series().diff().mean().seconds
    total_seconds = time_interval.total_seconds()
    estimated_rows_per_interval = total_seconds / mean_time_difference
    return int(estimated_rows_per_interval)

In [47]:
resolution_4_hours = calculate_resolution(df, time_interval=pd.Timedelta(hours=4))
resolution_6_hours = calculate_resolution(df, time_interval=pd.Timedelta(hours=6))
resolution_24_hours = calculate_resolution(df, time_interval=pd.Timedelta(hours=24))

resolution_4_hours

4

In [49]:
#Export weather data in different time_intervals (4,6,24)

weather_data_hourly_4 = aggregate_weather_data(df, time_interval= resolution_4_hours)
weather_data_hourly_4.to_pickle("data/weather_data_hourly_4.pickle")

weather_data_hourly_6 = aggregate_weather_data(df, time_interval= resolution_6_hours)
weather_data_hourly_6.to_pickle("data/weather_data_hourly_6.pickle")

weather_data_hourly_24 = aggregate_weather_data(df, time_interval = resolution_24_hours)
weather_data_hourly_24.to_pickle("data/weather_data_hourly_24.pickle")
